In [1]:
import pandas as pd
import numpy as np

# Annual

In [2]:
annual = pd.read_csv("inputs/annual_emissions.csv", encoding="latin1")

In [3]:
annual

,CNTR_NAME,ISO3,Gas,Component,Year,Data,Unit
0,Afghanistan,AFG,CH[4],Fossil,1830,0.018746,Tg~CH[4]~year^-1
1,Afghanistan,AFG,CH[4],Fossil,1831,0.018850,Tg~CH[4]~year^-1
2,Afghanistan,AFG,CH[4],Fossil,1832,0.018953,Tg~CH[4]~year^-1
3,Afghanistan,AFG,CH[4],Fossil,1833,0.019057,Tg~CH[4]~year^-1
4,Afghanistan,AFG,CH[4],Fossil,1834,0.019161,Tg~CH[4]~year^-1
...,...,...,...,...,...,...,...
369043,Zimbabwe,ZWE,N[2]*O,Total,2017,0.018637,Tg~N[2]*O~year^-1
369044,Zimbabwe,ZWE,N[2]*O,Total,2018,0.019151,Tg~N[2]*O~year^-1
369045,Zimbabwe,ZWE,N[2]*O,Total,2019,0.019338,Tg~N[2]*O~year^-1
369046,Zimbabwe,ZWE,N[2]*O,Total,2020,0.019186,Tg~N[2]*O~year^-1


In [4]:
annual = annual.drop(columns=["ISO3", "Unit"])

In [5]:
## Convert from terragrams to tonnes
annual["Data"] = annual["Data"] * 1E6

In [6]:
annual = annual.pivot(index=["CNTR_NAME", "Year", "Gas"], columns="Component", values="Data").reset_index()

In [7]:
annual

Component,CNTR_NAME,Year,Gas,Fossil,LULUCF,Total
0,ANNEXI,1830,CH[4],5.577567e+06,9.316435e+06,1.489400e+07
1,ANNEXI,1830,CO[2],8.912057e+04,NaN,8.912057e+04
2,ANNEXI,1830,N[2]*O,9.023376e+04,8.626969e+04,1.765035e+05
3,ANNEXI,1831,CH[4],5.623393e+06,9.419357e+06,1.504275e+07
4,ANNEXI,1831,CO[2],8.452541e+04,NaN,8.452541e+04
...,...,...,...,...,...,...
126331,Zimbabwe,2020,CO[2],1.060790e+04,8.182933e+03,1.879083e+04
126332,Zimbabwe,2020,N[2]*O,3.627974e+03,1.555766e+04,1.918563e+04
126333,Zimbabwe,2021,CH[4],2.239546e+05,2.616484e+05,4.856030e+05
126334,Zimbabwe,2021,CO[2],1.129611e+04,7.743253e+03,1.903937e+04


In [8]:
annual_co2 = annual.loc[annual["Gas"] == "CO[2]"]

In [9]:
annual_co2 = annual_co2.rename(columns={"Fossil":"annual_fossil_co2",
                                        "LULUCF":"annual_land_co2",
                                        "Total":"annual_co2"
                                       }
                              )

In [10]:
annual_co2 = annual_co2.drop(columns=["Gas"])

In [11]:
## Need to convert CO2 since it's in petagrams in original dataset
annual_co2["annual_fossil_co2"] = annual_co2["annual_fossil_co2"] * 1000
annual_co2["annual_land_co2"] = annual_co2["annual_land_co2"] * 1000
annual_co2["annual_co2"] = annual_co2["annual_co2"] * 1000

In [12]:
annual_ch4 = annual.loc[annual["Gas"] == "CH[4]"]

In [13]:
annual_ch4 = annual_ch4.rename(columns={"Fossil":"annual_fossil_ch4",
                                        "LULUCF":"annual_land_ch4",
                                        "Total":"annual_ch4"
                                       }
                              )

In [14]:
annual_ch4 = annual_ch4.drop(columns=["Gas"])

In [15]:
annual_n2o = annual.loc[annual["Gas"] == "N[2]*O"]

In [16]:
annual_n2o = annual_n2o.rename(columns={"Fossil":"annual_fossil_n2o",
                                        "LULUCF":"annual_land_n2o",
                                        "Total":"annual_n2o"
                                       }
                              )

In [17]:
annual_n2o = annual_n2o.drop(columns=["Gas"])

In [18]:
annual = pd.merge(annual_co2, annual_ch4)

In [19]:
annual = pd.merge(annual, annual_n2o)

In [20]:
## Conversion factors to get to CO2equivalents for CH4 and N2O
## All factors are from IPCC AR6

conversion_ch4_fossil = 29.8
conversion_ch4_land = 27.2
conversion_n2o = 273

In [21]:
## Convert gases to CO2-equivalents
annual["annual_fossil_ch4_CO2eq"] = annual["annual_fossil_ch4"] * conversion_ch4_fossil
annual["annual_land_ch4_CO2eq"] = annual["annual_land_ch4"] * conversion_ch4_land
annual["annual_ch4_CO2eq"] = annual["annual_fossil_ch4_CO2eq"] + annual["annual_land_ch4_CO2eq"]

annual["annual_fossil_n2o_CO2eq"] = annual["annual_fossil_n2o"] * conversion_n2o
annual["annual_land_n2o_CO2eq"] = annual["annual_land_n2o"] * conversion_n2o
annual["annual_n2o_CO2eq"] = annual["annual_n2o"] * conversion_n2o

In [22]:
annual["annual_ghg_CO2eq"] = annual["annual_co2"] + annual["annual_n2o_CO2eq"] + annual["annual_ch4_CO2eq"]
annual["annual_fossil_CO2eq"] = annual["annual_fossil_co2"] + annual["annual_fossil_n2o_CO2eq"] + annual["annual_fossil_ch4_CO2eq"]
annual["annual_land_CO2eq"] = annual["annual_land_co2"] + annual["annual_land_n2o_CO2eq"] + annual["annual_land_ch4_CO2eq"]


In [23]:
## Calculcate share of global total
annual_global = annual.loc[annual["CNTR_NAME"] == "GLOBAL"]

In [24]:
annual_global = annual_global.loc[:, ["Year", "annual_ch4", "annual_n2o", "annual_ghg_CO2eq"]]

In [25]:
annual_global = annual_global.rename(columns={"annual_ch4":"annual_ch4_global",
                                             "annual_n2o":"annual_n2o_global",
                                             "annual_ghg_CO2eq":"annual_ghg_global"
                                             }
                                    )

In [26]:
annual = pd.merge(annual, annual_global, how="outer")

In [27]:
annual

Component,CNTR_NAME,Year,annual_fossil_co2,annual_land_co2,annual_co2,annual_fossil_ch4,annual_land_ch4,annual_ch4,annual_fossil_n2o,annual_land_n2o,...,annual_ch4_CO2eq,annual_fossil_n2o_CO2eq,annual_land_n2o_CO2eq,annual_n2o_CO2eq,annual_ghg_CO2eq,annual_fossil_CO2eq,annual_land_CO2eq,annual_ch4_global,annual_n2o_global,annual_ghg_global
0,ANNEXI,1830,89120575.0,NaN,8.912058e+07,5.577567e+06,9.316435e+06,1.489400e+07,90233.764831,86269.689664,...,4.196185e+08,2.463382e+07,2.355163e+07,4.818544e+07,5.569246e+08,2.799659e+08,NaN,3.870508e+07,3.639436e+05,1.273576e+09
1,ANNEXII,1830,88001952.0,NaN,8.800195e+07,3.719275e+06,5.936784e+06,9.656059e+06,71428.368492,53198.224161,...,2.723149e+08,1.949994e+07,1.452312e+07,3.402306e+07,3.943399e+08,2.183363e+08,NaN,3.870508e+07,3.639436e+05,1.273576e+09
2,Afghanistan,1830,0.0,NaN,0.000000e+00,1.874618e+04,9.809356e+04,1.168397e+05,19.683200,730.977650,...,3.226781e+06,5.373514e+03,1.995569e+05,2.049304e+05,3.431711e+06,5.640098e+05,NaN,3.870508e+07,3.639436e+05,1.273576e+09
3,Albania,1830,0.0,NaN,0.000000e+00,7.646260e+03,1.128820e+04,1.893446e+04,50.978749,84.632071,...,5.348976e+05,1.391720e+04,2.310456e+04,3.702175e+04,5.719194e+05,2.417757e+05,NaN,3.870508e+07,3.639436e+05,1.273576e+09
4,Algeria,1830,0.0,NaN,0.000000e+00,1.324220e+04,2.267564e+04,3.591784e+04,81.954040,171.405260,...,1.011395e+06,2.237345e+04,4.679364e+04,6.916709e+04,1.080562e+06,4.169910e+05,NaN,3.870508e+07,3.639436e+05,1.273576e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40507,Venezuela,2021,79746599.0,2.468315e+07,1.044297e+08,1.447343e+06,1.070645e+06,2.517988e+06,6650.334000,40165.368000,...,7.225236e+07,1.815541e+06,1.096515e+07,1.278069e+07,1.894628e+08,1.246929e+08,6.476984e+07,3.649297e+08,1.087231e+07,5.459319e+10
40508,Viet Nam,2021,326013668.0,1.073186e+08,4.333322e+08,1.917947e+06,1.991719e+06,3.909666e+06,19073.565000,74999.292000,...,1.113296e+08,5.207083e+06,2.047481e+07,2.568189e+07,5.703437e+08,3.883756e+08,1.819681e+08,3.649297e+08,1.087231e+07,5.459319e+10
40509,Yemen,2021,12476597.0,-4.396800e+05,1.203692e+07,3.212172e+05,1.895279e+05,5.107452e+05,5971.964000,11016.988000,...,1.472743e+07,1.630346e+06,3.007638e+06,4.637984e+06,3.140233e+07,2.367922e+07,7.723117e+06,3.649297e+08,1.087231e+07,5.459319e+10
40510,Zambia,2021,7676180.0,4.694805e+07,5.462423e+07,3.041351e+05,4.376294e+05,7.417645e+05,4601.389000,48524.061000,...,2.096675e+07,1.256179e+06,1.324707e+07,1.450325e+07,9.009423e+07,1.799559e+07,7.209864e+07,3.649297e+08,1.087231e+07,5.459319e+10


In [28]:
annual["share_global_ch4"] = annual["annual_ch4"] / annual["annual_ch4_global"] * 100
annual["share_global_n2o"] = annual["annual_n2o"] / annual["annual_n2o_global"] * 100
annual["share_global_ghg"] = annual["annual_ghg_CO2eq"] / annual["annual_ghg_global"] * 100

In [29]:
annual = annual.drop(columns=["annual_ch4_global", "annual_n2o_global", "annual_ghg_global"])

# Cumulative

In [30]:
cumulative = pd.read_csv("inputs/cumulative_emissions.csv", encoding="latin1")

In [31]:
cumulative = cumulative.drop(columns=["ISO3", "Unit"])

In [32]:
## Convert from terragrams to tonnes
cumulative["Data"] = cumulative["Data"] * 1E9

In [33]:
cumulative = cumulative.pivot(index=["CNTR_NAME", "Year", "Gas"], columns="Component", values="Data").reset_index()

In [34]:
cumulative_co2 = cumulative.loc[cumulative["Gas"] == "CO[2]"]

In [35]:
cumulative_co2 = cumulative_co2.rename(columns={"Fossil":"cumulative_fossil_co2",
                                        "LULUCF":"cumulative_land_co2",
                                        "Total":"cumulative_co2"
                                       }
                              )

In [36]:
cumulative_co2 = cumulative_co2.drop(columns=["Gas"])

In [37]:
cumulative_ch4 = cumulative.loc[cumulative["Gas"] == "CH[4]"]

In [38]:
cumulative_ch4 = cumulative_ch4.rename(columns={"Fossil":"cumulative_fossil_ch4",
                                        "LULUCF":"cumulative_land_ch4",
                                        "Total":"cumulative_ch4"
                                       }
                              )

In [39]:
cumulative_ch4 = cumulative_ch4.drop(columns=["Gas"])

In [40]:
cumulative_n2o = cumulative.loc[cumulative["Gas"] == "N[2]*O"]

In [41]:
cumulative_n2o = cumulative_n2o.rename(columns={"Fossil":"cumulative_fossil_n2o",
                                        "LULUCF":"cumulative_land_n2o",
                                        "Total":"cumulative_n2o"
                                       }
                              )

In [42]:
cumulative_n2o = cumulative_n2o.drop(columns=["Gas"])

In [43]:
cumulative_ghg = cumulative.loc[cumulative["Gas"] == "3-GHG"]

In [44]:
cumulative_ghg = cumulative_ghg.rename(columns={"Fossil":"cumulative_fossil_ghg",
                                        "LULUCF":"cumulative_land_ghg",
                                        "Total":"cumulative_ghg"
                                       }
                              )

In [45]:
cumulative_ghg = cumulative_ghg.drop(columns=["Gas"])

In [46]:
cumulative = pd.merge(cumulative_co2, cumulative_ch4)
cumulative = pd.merge(cumulative, cumulative_n2o)
cumulative = pd.merge(cumulative, cumulative_ghg)

In [47]:
cumulative

Component,CNTR_NAME,Year,cumulative_fossil_co2,cumulative_land_co2,cumulative_co2,cumulative_fossil_ch4,cumulative_land_ch4,cumulative_ch4,cumulative_fossil_n2o,cumulative_land_n2o,cumulative_n2o,cumulative_fossil_ghg,cumulative_land_ghg,cumulative_ghg
0,ANNEXI,1851,1.988050e+08,1.459909e+09,1.658714e+09,2.845084e+08,2.810797e+08,5.655881e+08,3.591171e+07,5.208450e+07,8.799622e+07,5.192251e+08,1.793073e+09,2.312298e+09
1,ANNEXI,1852,4.063559e+08,2.936611e+09,3.342966e+09,5.718064e+08,5.642836e+08,1.136090e+09,7.225618e+07,1.052107e+08,1.774668e+08,1.050418e+09,3.606105e+09,4.656523e+09
2,ANNEXI,1853,6.235652e+08,4.447937e+09,5.071502e+09,8.653012e+08,8.496117e+08,1.714913e+09,1.090268e+08,1.593337e+08,2.683605e+08,1.597893e+09,5.456883e+09,7.054776e+09
3,ANNEXI,1854,8.787041e+08,5.982738e+09,6.861442e+09,1.186588e+09,1.137064e+09,2.323652e+09,1.462145e+08,2.143915e+08,3.606061e+08,2.211507e+09,7.334194e+09,9.545701e+09
4,ANNEXI,1855,1.138822e+09,7.522339e+09,8.661161e+09,1.509701e+09,1.426641e+09,2.936341e+09,1.838077e+08,2.703052e+08,4.541129e+08,2.832330e+09,9.219284e+09,1.205161e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36076,Zimbabwe,2017,7.534030e+08,1.456306e+09,2.209709e+09,5.464482e+08,7.281380e+08,1.274586e+09,5.208316e+07,3.047906e+08,3.568737e+08,1.351934e+09,2.489234e+09,3.841169e+09
36077,Zimbabwe,2018,7.651985e+08,1.464953e+09,2.230151e+09,5.538580e+08,7.275210e+08,1.281379e+09,5.303821e+07,3.090637e+08,3.621019e+08,1.372095e+09,2.501537e+09,3.873632e+09
36078,Zimbabwe,2019,7.763131e+08,1.473661e+09,2.249974e+09,5.607958e+08,7.263565e+08,1.287152e+09,5.400928e+07,3.133718e+08,3.673811e+08,1.391118e+09,2.513389e+09,3.904507e+09
36079,Zimbabwe,2020,7.869210e+08,1.481844e+09,2.268765e+09,5.678950e+08,7.266002e+08,1.294495e+09,5.499971e+07,3.176191e+08,3.726188e+08,1.409816e+09,2.526063e+09,3.935879e+09


# Temperature response

In [48]:
temp = pd.read_csv("inputs/temp_response.csv", encoding="latin1")

In [49]:
temp = temp.drop(columns=["ISO3", "Unit"])

In [50]:
temp = temp.pivot(index=["CNTR_NAME", "Year", "Gas"], columns="Component", values="Data").reset_index()

In [51]:
temp_co2 = temp.loc[temp["Gas"] == "CO[2]"]

In [52]:
temp_co2 = temp_co2.rename(columns={"Fossil":"temp_fossil_co2",
                                    "LULUCF":"temp_land_co2",
                                    "Total":"temp_co2"
                                       }
                              )

In [53]:
temp_co2 = temp_co2.drop(columns=["Gas"])

In [54]:
temp_ch4 = temp.loc[temp["Gas"] == "CH[4]"]

In [55]:
temp_ch4 = temp_ch4.rename(columns={"Fossil":"temp_fossil_ch4",
                                    "LULUCF":"temp_land_ch4",
                                    "Total":"temp_ch4"
                                    }
                              )

In [56]:
temp_ch4 = temp_ch4.drop(columns=["Gas"])

In [57]:
temp_n2o = temp.loc[temp["Gas"] == "N[2]*O"]

In [58]:
temp_n2o = temp_n2o.rename(columns={"Fossil":"temp_fossil_n2o",
                                    "LULUCF":"temp_land_n2o",
                                    "Total":"temp_n2o"
                                    }
                              )

In [59]:
temp_n2o = temp_n2o.drop(columns=["Gas"])

In [60]:
temp_ghg = temp.loc[temp["Gas"] == "3-GHG"]

In [61]:
temp_ghg = temp_ghg.rename(columns={"Fossil":"temp_fossil_ghg",
                                    "LULUCF":"temp_land_ghg",
                                    "Total":"temp_ghg"
                                    }
                              )

In [62]:
temp_ghg = temp_ghg.drop(columns=["Gas"])

In [63]:
temp_ghg

Component,CNTR_NAME,Year,temp_fossil_ghg,temp_land_ghg,temp_ghg
0,ANNEXI,1851,0.000234,0.000807,0.001041
4,ANNEXI,1852,0.000473,0.001624,0.002097
8,ANNEXI,1853,0.000720,0.002457,0.003177
12,ANNEXI,1854,0.000996,0.003303,0.004299
16,ANNEXI,1855,0.001275,0.004152,0.005427
...,...,...,...,...,...
152854,Zimbabwe,2017,0.000609,0.001121,0.001730
152858,Zimbabwe,2018,0.000618,0.001127,0.001744
152862,Zimbabwe,2019,0.000626,0.001132,0.001758
152866,Zimbabwe,2020,0.000635,0.001138,0.001772


In [64]:
temp = pd.merge(temp_co2, temp_ch4)
temp = pd.merge(temp, temp_n2o)
temp = pd.merge(temp, temp_ghg)

In [65]:
## Calculate share of global warming
temp_global = temp_ghg.loc[temp_ghg["CNTR_NAME"] == "GLOBAL"]
temp_global = temp_global.loc[:, ["Year", "temp_ghg"]]

In [66]:
temp_global = temp_global.rename(columns={"temp_ghg":"temp_ghg_global"
                                         }
                                )

In [67]:
temp = pd.merge(temp, temp_global, how="outer")

In [68]:
temp["share_global_warming"] = temp["temp_ghg"] / temp["temp_ghg_global"] * 100

In [69]:
temp = temp.drop(columns=["temp_ghg_global"])

# Combine

In [70]:
combined = pd.merge(annual, cumulative)
combined = pd.merge(combined, temp)

In [71]:
combined

Component,CNTR_NAME,Year,annual_fossil_co2,annual_land_co2,annual_co2,annual_fossil_ch4,annual_land_ch4,annual_ch4,annual_fossil_n2o,annual_land_n2o,...,temp_fossil_ch4,temp_land_ch4,temp_ch4,temp_fossil_n2o,temp_land_n2o,temp_n2o,temp_fossil_ghg,temp_land_ghg,temp_ghg,share_global_warming
0,ANNEXI,1851,198804976.0,1.459909e+09,1.658714e+09,7.384157e+06,1.111689e+07,1.850105e+07,131544.743472,190785.726245,...,1.281220e-04,1.265779e-04,2.546999e-04,1.617203e-05,2.345508e-05,3.962712e-05,2.338213e-04,8.074700e-04,1.041291e-03,60.018532
1,ANNEXII,1851,195071360.0,1.011704e+09,1.206775e+09,5.117125e+06,7.051977e+06,1.216910e+07,107504.702570,140831.102740,...,9.764406e-05,7.895629e-05,1.766004e-04,1.321656e-05,1.731369e-05,3.053026e-05,1.987066e-04,5.518680e-04,7.505746e-04,43.262039
2,Afghanistan,1851,0.0,3.529653e+06,3.529653e+06,2.099927e+04,1.099028e+05,1.309021e+05,23.794850,812.933250,...,2.018405e-07,9.649681e-07,1.166809e-06,2.925325e-09,9.994153e-08,1.028669e-07,2.047658e-07,2.654410e-06,2.859175e-06,0.164799
3,Albania,1851,0.0,8.060800e+05,8.060800e+05,8.627044e+03,1.282872e+04,2.145576e+04,65.504957,94.975233,...,8.598591e-08,1.208007e-07,2.067866e-07,8.053141e-09,1.167620e-08,1.972934e-08,9.403905e-08,4.954769e-07,5.895159e-07,0.033979
4,Algeria,1851,0.0,7.328000e+05,7.328000e+05,1.728660e+04,2.971497e+04,4.700158e+04,135.968810,220.366660,...,2.844764e-07,4.507993e-07,7.352757e-07,1.671593e-08,2.709175e-08,4.380767e-08,3.011923e-07,8.078910e-07,1.109083e-06,0.063926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36076,Venezuela,2021,79746599.0,2.468315e+07,1.044297e+08,1.447343e+06,1.070645e+06,2.517988e+06,6650.334000,40165.368000,...,2.436681e-03,1.301065e-03,3.737747e-03,4.792684e-05,2.656153e-04,3.135422e-04,6.006155e-03,4.477341e-03,1.048350e-02,0.650906
36077,Viet Nam,2021,326013668.0,1.073186e+08,4.333322e+08,1.917947e+06,1.991719e+06,3.909666e+06,19073.565000,74999.292000,...,2.122770e-03,2.352789e-03,4.475559e-03,7.204765e-05,3.468313e-04,4.188790e-04,4.277071e-03,6.564745e-03,1.084182e-02,0.673154
36078,Yemen,2021,12476597.0,-4.396800e+05,1.203692e+07,3.212172e+05,1.895279e+05,5.107452e+05,5971.964000,11016.988000,...,5.418637e-04,1.905301e-04,7.323938e-04,2.594360e-05,7.226957e-05,9.821318e-05,8.612392e-04,2.646972e-04,1.125936e-03,0.069908
36079,Zambia,2021,7676180.0,4.694805e+07,5.462423e+07,3.041351e+05,4.376294e+05,7.417645e+05,4601.389000,48524.061000,...,3.053843e-04,5.093227e-04,8.147070e-04,3.506382e-05,4.071919e-04,4.422557e-04,4.564803e-04,3.064380e-03,3.520860e-03,0.218606


# Standardise countries

In [72]:
countries = pd.read_csv("inputs/countries.csv", encoding="latin1")

In [73]:
combined = pd.merge(combined, countries)

In [74]:
combined = combined.drop(columns=["CNTR_NAME"])

In [75]:
combined = combined[combined["country"].notna()]

# Calculate per capita values

In [76]:
population = pd.read_csv("inputs/population.csv", encoding="latin1")

In [77]:
combined = pd.merge(combined, population, how="outer")

In [78]:
combined["annual_ch4_per_capita"] = combined["annual_ch4_CO2eq"] / combined["population"]
combined["annual_n2o_per_capita"] = combined["annual_n2o_CO2eq"] / combined["population"]
combined["annual_ghg_CO2eq_per_capita"] = combined["annual_ghg_CO2eq"] / combined["population"]

In [79]:
combined = combined.drop(columns=["population"])
combined = combined[combined["annual_co2"].notna()]
combined = combined.replace(np.inf, np.nan)

# Clean-up and export

In [80]:
combined = combined[ ["country"] + [ col for col in combined.columns if col != "country" ] ]

In [81]:
combined

,country,Year,annual_fossil_co2,annual_land_co2,annual_co2,annual_fossil_ch4,annual_land_ch4,annual_ch4,annual_fossil_n2o,annual_land_n2o,...,temp_fossil_n2o,temp_land_n2o,temp_n2o,temp_fossil_ghg,temp_land_ghg,temp_ghg,share_global_warming,annual_ch4_per_capita,annual_n2o_per_capita,annual_ghg_CO2eq_per_capita
0,Annex I,1851,198804976.0,1.459909e+09,1.658714e+09,7.384157e+06,1.111689e+07,1.850105e+07,131544.743472,190785.726245,...,0.000016,0.000023,0.000040,0.000234,0.000807,0.001041,60.018532,NaN,NaN,NaN
1,Annex I,1852,207550944.0,1.476702e+09,1.684253e+09,7.474989e+06,1.123066e+07,1.870565e+07,133129.907815,194601.307212,...,0.000033,0.000047,0.000080,0.000473,0.001624,0.002097,60.067434,NaN,NaN,NaN
2,Annex I,1853,217209248.0,1.511327e+09,1.728536e+09,7.594286e+06,1.134442e+07,1.893871e+07,134690.987939,198252.711181,...,0.000049,0.000072,0.000121,0.000720,0.002457,0.003177,60.184026,NaN,NaN,NaN
3,Annex I,1854,255138976.0,1.534801e+09,1.789940e+09,7.881019e+06,1.145819e+07,1.933921e+07,136218.747758,201677.186776,...,0.000066,0.000097,0.000162,0.000996,0.003303,0.004299,60.504172,NaN,NaN,NaN
4,Annex I,1855,260117847.0,1.539601e+09,1.799718e+09,7.965008e+06,1.157196e+07,1.953697e+07,137704.013643,204811.978227,...,0.000083,0.000122,0.000204,0.001275,0.004152,0.005427,60.716327,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35563,Zimbabwe,2017,9596071.0,1.016149e+07,1.975756e+07,2.133539e+05,2.498532e+05,4.632071e+05,3359.457000,15277.700000,...,0.000023,0.000137,0.000161,0.000609,0.001121,0.001730,0.114472,0.891727,0.344920,2.576042
35564,Zimbabwe,2018,11795478.0,8.647040e+06,2.044252e+07,2.157003e+05,2.566463e+05,4.723466e+05,3498.337000,15652.500000,...,0.000024,0.000139,0.000163,0.000618,0.001127,0.001744,0.113568,0.890810,0.347337,2.596256
35565,Zimbabwe,2019,11114607.0,8.708107e+06,1.982271e+07,2.177607e+05,2.614057e+05,4.791664e+05,3557.028000,15780.700000,...,0.000024,0.000141,0.000165,0.000626,0.001132,0.001758,0.112617,0.885695,0.343819,2.520509
35566,Zimbabwe,2020,10607897.0,8.182933e+06,1.879083e+07,2.208577e+05,2.618811e+05,4.827388e+05,3627.974000,15557.657000,...,0.000025,0.000143,0.000168,0.000635,0.001138,0.001772,0.111796,0.874602,0.334256,2.408044


In [82]:
combined.to_csv("output/Global warming contributions (Jones et al. 2022).csv", index=False)